# stata_session

> A class for representing a Stata session

Some parts adapted from the [stata_kernel version](https://github.com/kylebarron/stata_kernel/blob/master/stata_kernel/completions.py), limited for now to variables, globals, locals, scalars, matrices, and file names.

In [ ]:
#| default_exp stata_session
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.helpers import diverted_stata_output, run_as_program
from fastcore.basics import patch_to
from textwrap import dedent
import re

In [ ]:
#| export
class StataSession():
    def __init__(self):
        """"""
        self.matchall = re.compile(
            r"\A.*?"
            r"^%varlist%(?P<varlist>.*?)"
            r"%globals%(?P<globals>.*?)"
            r"%locals%(?P<locals>.*?)"
            r"%scalars%(?P<scalars>.*?)"
            r"%matrices%(?P<matrices>.*?)%end%", #"(\Z|---+\s*end)",
            flags=re.DOTALL + re.MULTILINE).match

        # Varlist-style matching; applies to most
        self.varlist = re.compile(r"(?:\s+)(\S+)", flags=re.MULTILINE)

        # file-style matching
        self.filelist = re.compile(r"[\r\n]{1,2}", flags=re.MULTILINE)

        # Clean line-breaks.
        self.varclean = re.compile(
            r"(?=\s*)[\r\n]{1,2}?^>\s", flags=re.MULTILINE).sub
        
        pre = (
            r'(cap(t|tu|tur|ture)?'
            r'|qui(e|et|etl|etly)?'
            r'|n(o|oi|ois|oisi|oisil|oisily)?)')
        kwargs = {'flags': re.MULTILINE}
        self.local_def_in = re.compile(
            r"^({0} )*(loc|loca|local)\s".format(pre),
            **kwargs,
        ).search

        self.parse_sreturn = re.compile(
            r'^\s*?(?:\ss\((?P<name>\w+)\) : \"(?P<value>.+)\"\s)', flags=re.MULTILINE
        ).findall
        
        #         # Match output from mata mata desc
#         self.matadesc = re.compile(
#             r"(\A.*?---+|---+[\r\n]*\Z)", flags=re.MULTILINE + re.DOTALL)

#         self.matalist = re.compile(
#             r"(?:.*?)\s(\S+)\s*$", flags=re.MULTILINE + re.DOTALL)

#         self.mataclean = re.compile(r"\W.*?(\b|$)")
#         self.matasearch = re.compile(r"(?P<kw>\w.*?(?=\W|\b|$))").search

        self.clear_suggestions()
#         self.suggestions = self.get_suggestions(kernel)
#         self.suggestions['magics'] = kernel.magics.available_magics
#         self.suggestions['magics_set'] = config.all_settings

    def clear_suggestions(self):
        self.suggestions = None

In [ ]:
from fastcore.test import test_eq

In [ ]:
test_instance = StataSession()
test_eq(bool(test_instance.local_def_in("sysuse auto")), False)
test_eq(bool(test_instance.local_def_in("loc auto=1")), True)
test_eq(bool(test_instance.local_def_in("qui n cap local auto=1")), True)
test_eq(bool(test_instance.local_def_in("list local auto")), False)

In [ ]:
#| export
@patch_to(StataSession)
def refresh_suggestions(self):
    self.suggestions = self.get_suggestions()
#     self.suggestions['magics_set'] = config.all_settings
#     self.globals = self.get_globals(kernel)

In [ ]:
#| export
def variable_names():
    from sfi import Data
    return [Data.getVarName(i) for i in range(Data.getVarCount())]

In [ ]:
#| eval: false
from nbstata.config import launch_stata

In [ ]:
#| eval: false
launch_stata(splash=False)

In [ ]:
#| eval: false
from pystata.stata import run

In [ ]:
#| eval: false
run("gen var1 = 1", quietly=True)
variable_names()

['var1']

In [ ]:
#| export
@patch_to(StataSession)
def _completions(self):
#     return dedent(f"""\
#     %varlist%
#     {' '.join(variable_names())}
#     %globals%
#     {' '.join(global_names())}
#     """
    return diverted_stata_output(dedent("""\
        local _temp_completions_while_local_ = 1
        while `_temp_completions_while_local_' {
        set more off
        set trace off
        if `"`varlist'"' != "" {
        local _temp_completions_varlist_loc_ `"`varlist'"'
        }
        syntax [varlist]
        disp "%varlist%"
        disp `"`varlist'"'
        macro drop _varlist __temp_completions_while_local_
        if `"`_temp_completions_varlist_loc_'"' != "" {
        local varlist `"`_temp_completions_varlist_loc_'"'
        macro drop __temp_completions_varlist_loc_
        }
        disp "%globals%"
        disp `"`:all globals'"'
        disp "%locals%"
        mata : invtokens(st_dir("local", "macro", "*")')
        disp "%scalars%"
        disp `"`:all scalars'"'
        disp "%matrices%"
        disp `"`:all matrices'"'
        disp "%end%"
        local _temp_completions_while_local_ = 0
        }
        macro drop _temp_completions_while_local_
    """), noecho=False)

In [ ]:
show_doc(StataSession._completions)

---

[source](https://github.com/hugetim/nbstata/blob/main/nbstata/stata_session.py#L81){target="_blank" style="float:right; font-size:smaller"}

### StataSession._completions

>      StataSession._completions ()

In [ ]:
#| eval: False
import pystata
pystata.stata.run("local varlist = 5")
pystata.stata.run("local varlist1 = 5")

In [ ]:
#| hide
#| eval: False
print(test_instance._completions())


. local _temp_completions_while_local_ = 1

. while `_temp_completions_while_local_' {
  2. set more off
  3. set trace off
  4. if `"`varlist'"' != "" {
  5. local _temp_completions_varlist_loc_ `"`varlist'"'
  6. }
  7. syntax [varlist]
  8. disp "%varlist%"
  9. disp `"`varlist'"'
 10. macro drop _varlist __temp_completions_while_local_
 11. if `"`_temp_completions_varlist_loc_'"' != "" {
 12. local varlist `"`_temp_completions_varlist_loc_'"'
 13. macro drop __temp_completions_varlist_loc_
 14. }
 15. disp "%globals%"
 16. disp `"`:all globals'"'
 17. disp "%locals%"
 18. mata : invtokens(st_dir("local", "macro", "*")')
 19. disp "%scalars%"
 20. disp `"`:all scalars'"'
 21. disp "%matrices%"
 22. disp `"`:all matrices'"'
 23. disp "%end%"
 24. local _temp_completions_while_local_ = 0
 25. }
%varlist%
var1
%globals%
S_level F1 F2 F7 F8 S_ADO S_StataMP S_StataSE S_CONSOLE S_FLAVOR S_OS S_OSDTL S
> _MACH
%locals%
  varlist varlist1
%scalars%

%matrices%

%end%

. macro drop _temp_co

In [ ]:
#| export
@patch_to(StataSession)
def get_suggestions(self):
    match = self.matchall(self._completions())
    suggestions = match.groupdict()
#         suggestions['mata'] = self._parse_mata_desc(suggestions['mata'])
#         suggestions['programs'] = self._parse_programs_desc(
#             suggestions['programs'])
    for k, v in suggestions.items():
#             if k in ['mata', 'programs']:
#                 continue
#             elif k in ['logfiles']:
#                 suggestions[k] = [
#                     f for f in self.filelist.split(v.strip()) if f]
#             else:
        suggestions[k] = self.varlist.findall(self.varclean('', v))
    #suggestions['locals'] = self.get_locals()
    return suggestions

In [ ]:
#| eval: false
test_instance.refresh_suggestions()
test_instance.suggestions

{'varlist': ['var1'],
 'globals': ['S_level',
  'F1',
  'F2',
  'F7',
  'F8',
  'S_ADO',
  'S_StataMP',
  'S_StataSE',
  'S_CONSOLE',
  'S_FLAVOR',
  'S_OS',
  'S_OSDTL',
  'S_MACH'],
 'locals': ['varlist', 'varlist1'],
 'scalars': [],
 'matrices': []}

In [ ]:
#| export
@patch_to(StataSession)
def get_locals(self):
    suggestions = self.get_suggestions() if self.suggestions is None else self.suggestions
    return suggestions['locals']
#     all_locals = """mata : invtokens(st_dir("local", "macro", "*")')"""
#     res = '\r\n'.join(
#         re.split(r'[\r\n]{1,2}', diverted_stata_output(all_locals)))
#     if res.strip():
#         return self.varlist.findall(self.varclean('', res))
#     else:
#         return []

In [ ]:
#| hide
#| eval: False
test_eq(test_instance.get_locals(), ['varlist', 'varlist1'])

In [ ]:
#| eval: false
from sfi import Macro
run(dedent("""
    local local1 = 1
    local local2 "two"
    local local3 `""3""' 
    """), quietly=True)
print(repr(Macro.getLocal("local1")))
print(repr(Macro.getLocal("local2")))
print(repr(Macro.getLocal("local3")))


'1'
'two'
'"3"'


In [ ]:
#| export
@patch_to(StataSession)
def get_local_dict(self):
    from sfi import Macro
    local_names = self.get_locals()
    return {n: Macro.getLocal(n) for n in local_names}

In [ ]:
#| eval: false
from nbstata.helpers import run_noecho

In [ ]:
from fastcore.test import test_eq

In [ ]:
#| eval: False
run_noecho('macro drop _all')
run_noecho('local test1 "blah blah"')
test_instance.clear_suggestions()
test_eq(test_instance.get_local_dict(), {'test1': 'blah blah'})
run_noecho('local test1 ""')

In [ ]:
#| hide
output = """
macros:
              s(test1) : "blah"
              s(test2) : "blah blah"

"""

In [ ]:
#| hide
test_instance.parse_sreturn("""
macros:
              s(test1) : "blah"
              s(test2) : "blah blah"

""")

[('test1', 'blah'), ('test2', 'blah blah')]

In [ ]:
#| export
@patch_to(StataSession)
def _local_dict_from_sreturn(self, sreturn_output):
    matches = self.parse_sreturn(sreturn_output)
    return {m[0]: m[1] for m in matches}

In [ ]:
#| hide
test_instance._local_dict_from_sreturn(output)

{'test1': 'blah', 'test2': 'blah blah'}

In [ ]:
#| export
def _run_as_program_w_locals_sreturned(std_code):
    sreturn_code = dedent("""\
        mata : st_local("all_locals", invtokens(st_dir("local", "macro", "*")'))
        foreach lname in `all_locals' {
            sreturn local `lname' "``lname''"
        }""")
    store_new_locals_code = ("sreturn clear\n" 
                             + std_code
                             + sreturn_code)                          
    run_as_program(store_new_locals_code, "sclass")

In [ ]:
#| export
def _locals_code_from_dict(preexisting_local_dict):
    local_defs = (f"""local {name} `"{preexisting_local_dict[name]}"'"""
                  for name in preexisting_local_dict)
    return "\n".join(local_defs)

In [ ]:
_locals_code_from_dict({'test1': 'blah', 'test2': 'blah blah'})

'local test1 `"blah"\'\nlocal test2 `"blah blah"\''

In [ ]:
#| hide
test_eq(_locals_code_from_dict({'test1': 'blah', 'test2': 'blah blah'}),
        """local test1 `"blah"\'\nlocal test2 `"blah blah"\'""")

In [ ]:
#| export
@patch_to(StataSession)
def _restore_locals_and_clear_sreturn(self):
    sreturn_output = diverted_stata_output("sreturn list") # one line to avoid clearing locals
    after_local_dict = self._local_dict_from_sreturn(sreturn_output)
    after_locals_code = _locals_code_from_dict(after_local_dict)
    if after_local_dict:
        after_locals_code += "\n" + "sreturn clear"
    run(after_locals_code, quietly=True)

In [ ]:
#| export
@patch_to(StataSession)
def run_as_prog_with_locals(self, std_code):
    """After `break_out_prog_blocks`, run noecho, inserting locals when needed"""
    from sfi import Macro
    from pystata.stata import run
    preexisting_local_dict = self.get_local_dict()
    locals_code = _locals_code_from_dict(preexisting_local_dict)
    if not self.local_def_in(std_code):
        run_as_program(f"""{locals_code}\n{std_code}""")
    else:
        _run_as_program_w_locals_sreturned(f"""{locals_code}\n{std_code}""")
        self._restore_locals_and_clear_sreturn()

In [ ]:
#| eval: false
run(dedent("""
    macro drop _all
    local local1 = 1
    local local2 "two"
    local local3 `""3""' 
    """), quietly=True)
test_instance.clear_suggestions()
test_instance.run_as_prog_with_locals("""disp `"`local1' `local2' `local3'"' """)


1 two "3"


In [ ]:
#| eval: false
code = '''\
local test1 "blah blah"
local test2 "blah"
'''
test_instance.clear_suggestions()
test_instance.run_as_prog_with_locals("""disp `"`local1' `local2' `local3'"' \n""" + code)
test_instance.get_local_dict()

1 two "3"



{'test2': 'blah',
 'test1': 'blah blah',
 'local1': '1',
 'local2': 'two',
 'local3': '"3"'}

In [ ]:
#| eval: false
test_instance.clear_suggestions()
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=test_instance.run_as_prog_with_locals)

1 two "3"
1 two "3" blah blah


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()